In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, get_json_object, collect_list, when, col

In [2]:
spark = SparkSession.builder \
    .appName("ol_works") \
    .getOrCreate()

23/08/05 09:07:59 WARN Utils: Your hostname, Stephanies-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.4.152 instead (on interface en0)
23/08/05 09:07:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/05 09:08:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
file_path = "./ol_dump_works_2023-07-31.txt"
ol_works_df = spark.read.text(file_path)
ol_works_df.show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                   

In [4]:
# {"title": "Qisas min al-hayah", "created": {"type": "/type/datetime", "value": "2009-12-11T01:57:19.964652"}, "last_modified": {"type": "/type/datetime", "value": "2009-12-11T01:57:19.964652"}, "latest_revision": 1, "key": "/works/OL10000315W", "authors": [{"type": "/type/author_role", "author": {"key": "/authors/OL3965336A"}}], "type": {"key": "/type/work"}, "revision": 1} 

{'title': 'Qisas min al-hayah',
 'created': {'type': '/type/datetime', 'value': '2009-12-11T01:57:19.964652'},
 'last_modified': {'type': '/type/datetime',
  'value': '2009-12-11T01:57:19.964652'},
 'latest_revision': 1,
 'key': '/works/OL10000315W',
 'authors': [{'type': '/type/author_role',
   'author': {'key': '/authors/OL3965336A'}}],
 'type': {'key': '/type/work'},
 'revision': 1}

In [5]:
split_col = split(ol_works_df["value"], "\t")

In [6]:
ol_works_df = ol_works_df.withColumn("type", split_col.getItem(0))
ol_works_df = ol_works_df.withColumn("work_key", split_col.getItem(1))
ol_works_df = ol_works_df.withColumn("revision", split_col.getItem(2))
ol_works_df = ol_works_df.withColumn("timestamp", split_col.getItem(3))
ol_works_df = ol_works_df.withColumn("json_data", split_col.getItem(4))



In [7]:
ol_works_df.show()

+--------------------+----------+------------------+--------+--------------------+--------------------+
|               value|      type|          work_key|revision|           timestamp|           json_data|
+--------------------+----------+------------------+--------+--------------------+--------------------+
|/type/work\t/work...|/type/work|/works/OL10000315W|       1|2009-12-11T01:57:...|{"title": "Qisas ...|
|/type/work\t/work...|/type/work|/works/OL10000836W|       4|2021-10-28T06:04:...|{"title": "PowerP...|
|/type/work\t/work...|/type/work|/works/OL10000889W|       2|2022-12-30T06:49:...|{"title": "La Rom...|
|/type/work\t/work...|/type/work|/works/OL10001324W|       3|2010-04-28T06:54:...|{"title": "Art no...|
|/type/work\t/work...|/type/work| /works/OL1000141W|       4|2012-07-09T00:02:...|{"title": "Das Re...|
|/type/work\t/work...|/type/work|/works/OL10001446W|       3|2010-04-28T06:54:...|{"title": "Rome 2...|
|/type/work\t/work...|/type/work|/works/OL10001662W|       3|201

In [8]:
ol_works_df.columns

['value', 'type', 'work_key', 'revision', 'timestamp', 'json_data']

In [9]:
json_works_data = ol_works_df.select("json_data")

In [12]:
# Extract the 'name' field from the JSON data using get_json_object
ol_works_df = ol_works_df.withColumn("title", get_json_object(ol_works_df["json_data"], "$.title"))
ol_works_df = ol_works_df.withColumn("author_key", get_json_object(ol_works_df["json_data"], "$.authors[0].author.key"))
ol_works_df = ol_works_df.withColumn("description", when(get_json_object(ol_works_df["json_data"], "$.description").isNotNull(),
                                                         get_json_object(ol_works_df["json_data"], "$.description"))
                                              .otherwise(None))
ol_works_df.columns

['value',
 'type',
 'work_key',
 'revision',
 'timestamp',
 'json_data',
 'title',
 'author_key',
 'description']

In [14]:
works_key_df = ol_works_df.select("work_key","title","author_key","description")
works_key_df.show()

+------------------+--------------------+-------------------+--------------------+
|          work_key|               title|         author_key|         description|
+------------------+--------------------+-------------------+--------------------+
|/works/OL10000315W|  Qisas min al-hayah|/authors/OL3965336A|                null|
|/works/OL10000836W|     PowerPoint 2000|/authors/OL3965931A|                null|
|/works/OL10000889W|La Rome antique d...|/authors/OL3965988A|                null|
|/works/OL10001324W|         Art nouveau|/authors/OL3966449A|                null|
| /works/OL1000141W|Das Reich der Ins...|/authors/OL4740632A|                null|
|/works/OL10001446W|           Rome 2002|/authors/OL3966612A|                null|
|/works/OL10001662W|Partir etudier au...|/authors/OL3966842A|                null|
|/works/OL10001801W|Le p'tit dico de ...|/authors/OL3966991A|                null|
|/works/OL10002285W|MALESTROIT HIER E...|/authors/OL3967610A|                null|
|/wo

In [18]:
# Why description won't work lol
null_count = works_key_df.filter(col("description").isNull()).count()
works_count = works_key_df.count()
percentage_null = (null_count / works_count)*100

print("Number of null values in 'description' column:", null_count)
print("Total number of works listed:", works_count)
print("Percentage with null description:", percentage_null)


Number of null values in 'description' column: 31972812
Total number of works listed: 33235817
Percentage with null description: 96.19986775110718


In [ ]:
output_path = "./data/works_key_parquet"

works_key_df = works_key_df.repartition(1)

# Write the DataFrame to Parquet format
works_key_df.write.parquet(output_path)